In [110]:
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import scipy.stats as stats

plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Yu Gothic', 'IPAexGothic', 'Noto Sans CJK JP']
plt.rcParams['axes.unicode_minus'] = False

In [111]:
#データを整理
df23_24 = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】プレイバイプレイ_23-24シーズン.csv')
df24_25 = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】プレイバイプレイ_24-25シーズン.csv')
df_box = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】ボックススコア.csv')
df_game = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】試合データ.csv')
df_team = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】チームマスタ.csv')
df_cluster = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/analysis/B1_clusters_2024-25.csv', header=1)
df_player = pd.read_csv('/Users/nakamurawataru/Documents/学校/研究室/SDSC/03.バスケ/6月送付分/【2025年度】選手マスタ.csv')
df_all = pd.concat([df23_24, df24_25], ignore_index=True)

df_box['プレイタイム_秒'] = (
    df_box['プレイタイム']
    .astype(str)                               # 念のため文字列化
    .str.replace('：', ':', regex=False)       # 全角コロン対策
    .apply(lambda x: int(x.split(':')[0]) * 60 + int(x.split(':')[1]) if ':' in x else 0)
)

#リーグの試合IDを抽出
cupID = [500, 507]
df_gameID_cupID = df_game[df_game['カップID'].isin(cupID)]
df_gameID_cupID = df_gameID_cupID['試合ID'].unique()

cupID24_25 = [507]
df_gameID_cupID24_25 = df_game[df_game['カップID'].isin(cupID24_25)]
df_gameID_cupID24_25 = df_gameID_cupID24_25['試合ID'].unique()

#試合IDからデータを抽出
df23_24 = df23_24[df23_24['試合ID'].isin(df_gameID_cupID24_25)]
df24_25 = df24_25[df24_25['試合ID'].isin(df_gameID_cupID24_25)]
df_game = df_game[df_game['試合ID'].isin(df_gameID_cupID24_25)]
df_box24_25 = df_box[df_box['試合ID'].isin(df_gameID_cupID24_25)]

# 試合ID、チームIDごとにデータを整理
df_box_gameid = df_box.drop(columns=['ホームアウェイ','選手ID','背番号','選手名','スターティングフラグ','プレイタイム'])
df_box_gameid_sum = df_box_gameid[df_box_gameid['ピリオド区分'] == 18].groupby(['試合ID','チームID']).sum(numeric_only=True).reset_index()

# 試合IDごとに勝敗を割り当て
# 1. 試合IDごとに得点の最大値を取得
max_scores = df_box_gameid_sum.groupby('試合ID')['得点'].transform('max')
# 2. 自チームの得点が最大なら1（勝ち）、そうでなければ0（負け）
df_box_gameid_sum['勝敗'] = (df_box_gameid_sum['得点'] == max_scores).astype(int)

# 試合データから失点列を作成（より効率的）
df_game_long = pd.concat([
    df_game[['試合ID', 'ホームチームID', 'アウェイ得点']].rename(
        columns={'ホームチームID': 'チームID', 'アウェイ得点': '失点'}
    ),
    df_game[['試合ID', 'アウェイチームID', 'ホーム得点']].rename(
        columns={'アウェイチームID': 'チームID', 'ホーム得点': '失点'}
    )
], ignore_index=True)

# df_box_gameid_sum に失点を結合
df_box_gameid_sum = df_box_gameid_sum.merge(
    df_game_long[['試合ID', 'チームID', '失点']],
    on=['試合ID', 'チームID'],
    how='left'
)

# 安全に成功率を計算するヘルパー
def safe_rate(df, num_col, den_col, out_col, fillna=0):
    if num_col not in df.columns or den_col not in df.columns:
        print(f"警告: 列がありません: {num_col} または {den_col}")
        return
    # 数値化（非数値は NaN）
    df[num_col] = pd.to_numeric(df[num_col], errors='coerce')
    df[den_col] = pd.to_numeric(df[den_col], errors='coerce')
    # 0 を NaN にしてゼロ除算を回避
    denom = df[den_col].replace({0: np.nan})
    # 計算（結果を float に強制）
    df[out_col] = (df[num_col] / denom).astype(float)
    # 数値化の再適用（万が一の非数を排除）
    df[out_col] = pd.to_numeric(df[out_col], errors='coerce')
    # 現実的な範囲に収める（負や 1 より大きい値が出た場合にクリップ）
    df[out_col] = df[out_col].clip(lower=0.0, upper=1.0)
    # オプションで NaN を置換
    if fillna is not None:
        df[out_col] = df[out_col].fillna(fillna)

# 成功率列を安全に作成
safe_rate(df_box_gameid_sum, '2P成功', '2P試投', '2P成功率')
safe_rate(df_box_gameid_sum, '3P成功', '3P試投', '3P成功率')
safe_rate(df_box_gameid_sum, 'フリースロー成功', 'フリースロー試投', 'フリースロー成功率')

mask = df_box_gameid_sum['試合ID'].isin(df_gameID_cupID24_25)
df_box_gameid_sum = df_box_gameid_sum.loc[mask].copy()

df_box24_25_gameid_sum = df_box_gameid_sum[df_box_gameid_sum['試合ID'].isin(df_gameID_cupID24_25)]

# # ポジション抽出（大文字化して単語境界で SF または SG をマッチ）
# df_sf_sg = df_players[df_players['ポジション'].astype(str).str.upper().str.contains(r'\b(?:SF|SG)\b', na=False)]

# # 選手ID を重複除去してリスト化（必要なら文字列化）
# ids = df_sf_sg['選手ID'].dropna().unique().tolist()

# # df_box24_25 の選手ID 列と型が異なる可能性がある場合は同型化してから isin
# df_sf_sg_player = df_box24_25[df_box24_25['選手ID'].isin(ids)]
# df_sf_sg_player_nation = df_players[df_players['選手ID'].isin(ids)]

In [136]:
# アクション列を数値化（非数は NaN に）
df24_25['アクション1'] = pd.to_numeric(df24_25['アクション1'], errors='coerce')

# 同一試合内の「次の行」のアクションを取得
next_action = df24_25['アクション1'].shift(-1)

# 現行が 1 かつ 次が 12 の場合、現行行の A3P を 1 に（それ以外は 0）
df24_25['A3P'] = (((df24_25['アクション1'] == 1) & (next_action == 12)).astype(int))

df_all_byplayer = df24_25.groupby('選手ID1', as_index=False)['A3P'].sum().rename(columns={'A3P':'A3P_sum','選手ID1':'選手ID'})

df_players = df_box24_25.copy()
df_players = df_players[['選手ID','プレイタイム_秒','3P成功','3P試投']]
df_players = df_players.groupby('選手ID').sum()

df_players_new = df_players.merge(df_all_byplayer, on='選手ID', how='left').fillna({'A3P_sum': 0})
df_players_new['3FG%'] = df_players_new['3P成功'] / df_players_new['3P試投']
df_players_new['A3P_40min'] = df_players_new['A3P_sum'] / df_players_new['プレイタイム_秒'] * 40 * 60
df_players_new['C&S'] = 2.5 * df_players_new['3FG%'] + df_players_new['A3P_40min']

df_players_sorted = (
    df_players_new
    .sort_values('C&S', ascending=False)
    .reset_index()  # 選手IDを保持したい場合は reset_index(drop=True) に変更可
)

df_players_sorted = df_players_sorted[df_players_sorted['3P試投'] > 100].reset_index(drop=True)
df_players_sorted = df_players_sorted.merge(
    df_player[['選手ID', '選手名']].drop_duplicates(subset='選手ID'),
    on='選手ID',
    how='left'
)

df_players_sorted = df_players_sorted[['C&S','選手名']]
df_players_sorted.to_csv('df_players_sorted.csv', index=False, encoding='utf-8-sig')

In [134]:
df_players_sorted

,index,選手ID,プレイタイム_秒,3P成功,3P試投,A3P_sum,3FG%,A3P_40min,C&S,選手名
0,337,5100000011,146390,250,624,123.0,0.400641,2.016531,3.018134,津屋 一球
1,5,8468,115056,202,486,92.0,0.415638,1.919065,2.958160,田口 成浩
2,182,32982,111999,180,448,85.0,0.401786,1.821445,2.825909,角野 亮伍
3,80,9405,92706,138,316,65.0,0.436709,1.682739,2.774511,森川 正明
4,13,8487,143251,254,584,93.0,0.434932,1.558104,2.645433,辻 直人
...,...,...,...,...,...,...,...,...,...,...
206,111,15834,113499,32,156,16.0,0.205128,0.338329,0.851149,半澤 凌太
207,101,12634,131310,30,132,15.0,0.227273,0.274160,0.842342,エリック・ジェイコブセン
208,264,51000317,213268,22,110,10.0,0.200000,0.112534,0.612534,ヨーリ・チャイルズ
209,140,25267,77572,14,104,7.0,0.134615,0.216573,0.553111,ハビエル・カーター


In [114]:
print(df_players_new['A3P_40min'].mean())
print(df_players_new['3FG%'].mean())

0.7360935740425854
0.305814465067568


In [115]:
df_players

,プレイタイム_秒,3P成功,3P試投
選手ID,,,
8447,208232,172,520
8452,19309,2,26
8456,10072,4,18
8463,78853,78,188
8467,235267,248,832
...,...,...,...
5100000069,211827,208,660
5100000079,47958,22,98
5100000081,142635,62,198


In [116]:
df_player

,カップID,チームID,選手ID,背番号,選手名,ポジション,国籍,生年月日,身長,体重,在籍フラグ
0,500,692,8462,0,小林 遥太,PG,JAPAN,19910912,178,77,1
1,500,692,22401,6,岡田 泰希,PG/SG,JAPAN,19990728,176,76,1
2,500,692,10263,7,澤邉 圭太,SG,JAPAN,19950305,185,87,1
3,500,692,40169,8,デボーン・ワシントン,PF,UNITED STATES,19890329,203,102,0
4,500,692,33036,9,ヤン ジェミン,SF,"KOREA, REPUBLIC OF",19990622,201,93,1
...,...,...,...,...,...,...,...,...,...,...,...
1629,513,2891,10844,21,満田 丈太郎,SG/SF,JAPAN,19940418,188,83,1
1630,513,2891,9283,22,小阪 彰久,C/PF,JAPAN,19920222,198,102,1
1631,513,2891,22522,30,ペリー・エリス,PF,UNITED STATES,19930914,202,100,1
1632,513,2891,18442,34,ベンジャミン・ローソン,C/PF,UNITED KINGDOM,19950612,216,118,1
